# Japanese Translation with NLP


## Introduction
In this noteboook, a recurrent deep neural network will be built to translate Japanese to English.
The completed pipeline will accept Japanese text as input and return the English translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [24]:
import collections

#import helper
import os
import numpy as np
#import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [25]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12813410616380975237
]


https://github.com/odashi/small_parallel_enja

## Dataset
The corpus for this project is provided by Stanford University's Japanese English Subtitle Corpus (https://nlp.stanford.edu/projects/jesc/). THe full raw data set contains over 3 million sample sentences of Japanese and their English translation. The authors of the corpus have take the liberty to pre-tokenize the testing and training data set for ease of use.


@ARTICLE{pryzant_jesc_2017,
   author = {{Pryzant}, R. and {Chung}, Y. and {Jurafsky}, D. and {Britz}, D.},
    title = "{JESC: Japanese-English Subtitle Corpus}",
  journal = {ArXiv e-prints},
archivePrefix = "arXiv",
   eprint = {1710.10639},
 keywords = {Computer Science - Computation and Language},
     year = 2017,
    month = oct,
}                
    

### Load Data
The data is located in `data/*`. The tokenized training data sets are in `data/tokenzed/train.en` and `data/tokenized/train.ja` which contain the English sentences with their Japanese translations 

In [26]:
def load_data(path):
    """
    Load dataset
    """
    input_file = os.path.join(path)
    with open(input_file, "r", encoding="utf-8") as f:
        data = f.read()
    return data.split('\n')


In [27]:
# Load English data
#en_sent_raw = load_data('data/raw/en')
en_sent_token_train = load_data('data/odashi/train.en')

# Load Japanese data
#ja_sent_raw = load_data('data/raw/ja')
ja_sent_token_train = load_data('data/odashi/train.ja')

print('Dataset Loaded')


Dataset Loaded


In [28]:
#for sample_i in range(3):
#    print('Japanese Sentence Line {}:  {}'.format(sample_i + 1, ja_sent_raw[sample_i]))
#    print('English Sentence Line {}:  {}'.format(sample_i + 1, en_sent_raw[sample_i]))
    
print("----------------------")
for sample_i in range(3):    
    print('Japanese Sentence Line {}:  {}'.format(sample_i + 1, ja_sent_token_train[sample_i]))
    print('English Sentence Line {}:  {}'.format(sample_i + 1, en_sent_token_train[sample_i]))
    

----------------------
Japanese Sentence Line 1:  誰 が 一番 に 着 く か 私 に は 分か り ま せ ん 。
English Sentence Line 1:  i can 't tell who will arrive first .
Japanese Sentence Line 2:  多く の 動物 が 人間 に よ っ て 滅ぼ さ れ た 。
English Sentence Line 2:  many animals have been destroyed by men .
Japanese Sentence Line 3:  私 は テニス 部員 で す 。
English Sentence Line 3:  i 'm in the tennis club .


### Vocabulary
Since Japanese does not take into account spaces between word or vocabulary. Simple split functions will not be able to be applied to distinguish words.  The tokenized dataset provided in the JESC adds the space between words, as well as underscores to distinguish beginning of words with characters.

In [29]:
en_words_counter = collections.Counter([word for sentence in en_sent_token_train for word in sentence.split()])
ja_words_counter = collections.Counter([word for sentence in ja_sent_token_train for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in en_sent_token_train for word in sentence.split()])))
print('{} unique English words.'.format(len(en_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*en_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Japanese words.'.format(len([word for sentence in ja_sent_token_train for word in sentence.split()])))
print('{} unique Japanese words.'.format(len(ja_words_counter)))
print('10 Most common words in the Japanese dataset:')
print('"' + '" "'.join(list(zip(*ja_words_counter.most_common(10)))[0]) + '"')

391047 English words.
6634 unique English words.
10 Most common words in the English dataset:
"." "the" "i" "to" "you" "is" "he" "a" "?" "in"

565618 Japanese words.
8774 unique Japanese words.
10 Most common words in the Japanese dataset:
"。" "は" "い" "に" "た" "を" "の" "て" "で" "な"


## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [30]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer() #tokenize to the word level
    x_tk.fit_on_texts(x) #tokenize text x    
    
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = en_sent_token_train[:3]
#    [
#    'The quick brown fox jumps over the lazy dog .',
#    'By Jove , my quick study of lexicography won a prize .',
#    'This is a short sentence .']

text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'i': 1, 'can': 2, "'t": 3, 'tell': 4, 'who': 5, 'will': 6, 'arrive': 7, 'first': 8, 'many': 9, 'animals': 10, 'have': 11, 'been': 12, 'destroyed': 13, 'by': 14, 'men': 15, "'m": 16, 'in': 17, 'the': 18, 'tennis': 19, 'club': 20}

Sequence 1 in x
  Input:  i can 't tell who will arrive first .
  Output: [1, 2, 3, 4, 5, 6, 7, 8]
Sequence 2 in x
  Input:  many animals have been destroyed by men .
  Output: [9, 10, 11, 12, 13, 14, 15]
Sequence 3 in x
  Input:  i 'm in the tennis club .
  Output: [1, 16, 17, 18, 19, 20]


### Padding (

In [31]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    if length is None:
        length = max([len(sentence) for sentence in x]) # set length equal to max sequence when there is no designated length value
    padded_sequence = pad_sequences(x, maxlen=length, padding='post')  # add 0's after sequence until sequence reaches max length
    return padded_sequence

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 3 4 5 6 7 8]
  Output: [1 2 3 4 5 6 7 8]
Sequence 2 in x
  Input:  [ 9 10 11 12 13 14 15]
  Output: [ 9 10 11 12 13 14 15  0]
Sequence 3 in x
  Input:  [ 1 16 17 18 19 20]
  Output: [ 1 16 17 18 19 20  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [35]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x) # tokenize input x
    preprocess_y, y_tk = tokenize(y) # tokenize input y

    preprocess_x = pad(preprocess_x) # pad input tokenized x
    preprocess_y = pad(preprocess_y) # pad input tokenized y

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1) # add third dimension '1'

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_ja_sentences, preproc_en_sentences, ja_tokenizer, en_tokenizer =\
    preprocess(ja_sent_token_train, en_sent_token_train)
    
max_en_sequence_length = preproc_en_sentences.shape[1]
max_ja_sequence_length = preproc_ja_sentences.shape[1]
en_vocab_size = len(en_tokenizer.word_index)
ja_vocab_size = len(ja_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_en_sequence_length)
print("Max Japanese sentence length:", max_ja_sequence_length)
print("English vocabulary size:", en_vocab_size)
print("Japanese vocabulary size:", ja_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max Japanese sentence length: 16
English vocabulary size: 6547
Japanese vocabulary size: 8774


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [36]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [37]:
def simple_model(input_shape, output_sequence_length, ja_vocab_size, en_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    #: Build the layers
    learning_rate = 0.001
    input_seq = Input(shape = (input_shape[1:])) 
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(ja_vocab_size))(rnn)
    
    model = Model(inputs = input_seq, outputs = Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_ja_sentences, max_en_sequence_length)
print(tmp_x.shape)
tmp_x = tmp_x.reshape((-1, preproc_en_sentences.shape[-2], 1))
print(tmp_x.shape)

print(preproc_en_sentences.shape)

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_en_sequence_length,
    ja_vocab_size,
    en_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_en_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], en_tokenizer))

(50000, 15)
(50000, 15, 1)
(50000, 15, 1)
Train on 40000 samples, validate on 10000 samples
Epoch 1/10


InvalidArgumentError: Incompatible shapes: [15360] vs. [1024,15]
	 [[Node: metrics_3/acc/Equal = Equal[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](metrics_3/acc/Reshape, metrics_3/acc/Cast)]]

### Model 2: Embedding (IMPLEMENTATION)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [38]:
def embed_model(input_shape, output_sequence_length, ja_vocab_size, en_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 0.001
    
    input_seq = Input(shape=(input_shape[1:])) # assign the input sequence
    
    # Embedding() : https://keras.io/layers/embeddings/
    embed = Embedding(input_dim=ja_vocab_size, output_dim=en_vocab_size,  input_length=input_shape[1])(input_seq)  # create an embedding layer to represent each word as a vecto

    rnn = GRU(64, return_sequences=True, activation = 'tanh')(embed)
    logits = TimeDistributed(Dense(en_vocab_size))(rnn)
    
    model = Model(inputs = input_seq, outputs = Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model
#tests.test_embed_model(embed_model)


# Reshape the input
tmp_x = pad(preproc_ja_sentences, max_en_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_en_sentences.shape[-2]))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_en_sequence_length,
    ja_vocab_size,
    en_vocab_size)

embed_rnn_model.fit(tmp_x, preproc_en_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], en_tokenizer))

Train on 40000 samples, validate on 10000 samples
Epoch 1/20


InvalidArgumentError: Incompatible shapes: [15360] vs. [1024,15]
	 [[Node: metrics_4/acc/Equal = Equal[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](metrics_4/acc/Reshape, metrics_4/acc/Cast)]]

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [18]:
def bd_model(input_shape, output_sequence_length, ja_vocab_size, en_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    input_seq = Input(shape=(input_shape[1:]))
    
    # Wrappers https://keras.io/layers/wrappers/  
    bdrnn = Bidirectional(GRU(64, return_sequences=True, activation="tanh"), input_shape=input_shape[1:])(input_seq)
    
    logits = TimeDistributed(Dense(en_vocab_size))(bdrnn)

    model = Model(inputs=input_seq, outputs=Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_ja_sentences, max_en_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_en_sentences.shape[-2],1)) # need to reshape input


# Train the neural network
bdrnn_model = bd_model(
    tmp_x.shape,
    max_en_sequence_length,
    ja_vocab_size,
    en_vocab_size)
bdrnn_model.fit(tmp_x, preproc_en_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bdrnn_model.predict(tmp_x[:1])[0], en_tokenizer))

Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 23s - loss: 7.4310 - acc: 0.1683 - val_loss: nan - val_acc: 0.0690
Epoch 2/10
40000/40000 [==============================] - 22s - loss: 4.4867 - acc: 0.2257 - val_loss: nan - val_acc: 0.3207
Epoch 3/10
40000/40000 [==============================] - 22s - loss: 3.9308 - acc: 0.3330 - val_loss: nan - val_acc: 0.3462
Epoch 4/10
40000/40000 [==============================] - 22s - loss: 3.7785 - acc: 0.3467 - val_loss: nan - val_acc: 0.3470
Epoch 5/10
40000/40000 [==============================] - 22s - loss: 3.7206 - acc: 0.3460 - val_loss: nan - val_acc: 0.3459
Epoch 6/10
40000/40000 [==============================] - 22s - loss: 3.6880 - acc: 0.3458 - val_loss: nan - val_acc: 0.3465
Epoch 7/10
40000/40000 [==============================] - 22s - loss: 3.6661 - acc: 0.3462 - val_loss: nan - val_acc: 0.3466
Epoch 8/10
40000/40000 [==============================] - 22s - loss: 3.648

In [40]:
def model_final(input_shape, output_sequence_length, ja_vocab_size, en_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Implement
    learning_rate = 0.001
    
    input_seq = Input(shape=(input_shape[1:]))

    # Embedding() : https://keras.io/layers/embeddings/
    embed = Embedding(input_dim=ja_vocab_size, output_dim=en_vocab_size,  input_length=output_sequence_length)(input_seq)  # create an embedding layer to represent each word as a vecto

    # Wrappers https://keras.io/layers/wrappers/     # GRU Gated Recurrent Unit
    bdrnn = Bidirectional(GRU(64, return_sequences=True, activation="tanh"), input_shape=input_shape[1:])(embed)

    logits = TimeDistributed(Dense(en_vocab_size, activation = 'softmax'))(bdrnn)
    
    model = Model(inputs = input_seq, outputs = logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    return model
#tests.test_model_final(model_final)


print('Final Model Loaded')
# Train the final model
# Reshaping the input
tmp_x = pad(preproc_ja_sentences, max_en_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_en_sentences.shape[-2]))

# Train the neural network
final_model = model_final(
    tmp_x.shape,
    max_en_sequence_length,
    ja_vocab_size,
    en_vocab_size)
final_model.fit(tmp_x, preproc_en_sentences, batch_size=1024, epochs=20, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_model.predict(tmp_x[:1])[0], en_tokenizer))

Final Model Loaded
Train on 40000 samples, validate on 10000 samples
Epoch 1/20


InvalidArgumentError: Incompatible shapes: [15360] vs. [1024,15]
	 [[Node: metrics_5/acc/Equal = Equal[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"](metrics_5/acc/Reshape, metrics_5/acc/Cast)]]